## Trace Simulator Demo

Notebook to generate toy data using the DELight TraceSimulator package as input

To install helix, run pip install -e /path/to/helix/repo 

In [ ]:
import helix as hx
import strax as sx  
from matplotlib import pyplot as plt
import numpy as np  
import os
import shutil
from glob import glob

### Generate toy data using trace simulator 

The trace simulator toy data generator can produce data for different scenarios. 
- Provide a single event energy, recoil type, and event rate. The generated data will have a poissonian number of waveforms at random times throughout the stream according to the run duration and the event rate.
- Provide a range of energies in the format [E1,E2], where energies will be uniformly drawn bewteen the two values. Also provide recoil type and event rate.
- Provide a list of specific energies to use [E1,E2,E3,...,En]. Also provide recoil type and event rate
- Provide the output of a geant4 simulation. Also provide event rate. The recoil type is taken from the simulation.

Let's generate data with 500eV NR recoils. 

In [ ]:
run_id = 'run_5keV_NR' 
duration = 10  # seconds
raw_data_dir = 'tracesimulator_data'  # to save the raw toy data
helix_data_dir = 'test_helix_data'  # to save the run metadata
baseline_step = 0  # add a baseline equal to baseline_step*channel_index to each channel  

energy = 5000 #eV
recoil_type = 'NR'

In [ ]:
# remove helix data corresponding to this run_id, if it exists
for path in glob(f'{helix_data_dir}/*'):
    if os.path.isdir(path):
        shutil.rmtree(path)
    else:
        os.remove(path)

hx.generate_tracesimulator_data(run_id,duration,event_rate=3,event_energy=energy,overwrite=True,recoil_type=recoil_type,directory=raw_data_dir,helix_data_dir =helix_data_dir )

### Process the data
Once the data have been generated, let's process them with helix 

In [ ]:
# creating context, registering storage and plugins
context = sx.Context(storage=[sx.DataDirectory(helix_data_dir, provide_run_metadata=True), ],
                     register=[hx.ToyDataRawRecords,
                               hx.QPTriggers, hx.UVTriggers,
                               hx.Events, hx.NoiseEvents,
                               hx.NoisePSDs, hx.FitResults,
                               hx.EventsBasics])    # all the plugins required for getting fit_results

# creating a dictionary of plugins' options that we want to modify.
config = {'run_metadata_directory': helix_data_dir,      # for the hx.ToyDataRawRecords plugin
          'noise_events_random_seed': 1,                 # for the hx.NoiseEvents plugin
          'daq_data_directory': raw_data_dir,
          'trigger_threshold':100,
          'templates_path':'plugins/event_rqs/signalformation_templates.npy'}                 # for the hx.NoiseEvents plugin

# passing the settings to the plugins. Strax finds which plugins take these options automatically
context.set_config(config)

# running the processing (or retrieving the data, if it was processed earlier)
psds = context.get_array(run_id, 'noise_psds')  # Helix needs to prebuild PSDs before calculating fit_results 
fit_results = context.get_array(run_id, 'fit_results')  # hx.FitResults plugin provides this data type

# fit_results is a structured numpy array of events containing fields described in the FitResults documentation
# one can work with it as a dictionary on numpy arrays, or as a numpy array of dictionaries. Works both ways. E.g.
# fit_results['sum_uv_amplitude'][:100]  # amplitudes of UV signals in the sum of all channels in the first 100 events
# fit_results[:100]['sum_uv_amplitude']  # same

## Results of Processing

In [ ]:
plt.scatter(fit_results['triggered_uv_amplitude'], fit_results['triggered_qp_amplitude'])
plt.xlabel('UV signal amplitude')
plt.ylabel('QP signal amplitude')
plt.xlim(-1000, 20000)
plt.ylim(-1000, 40000)
plt.show()

In [ ]:
events = context.get_array(run_id, 'events')

In [ ]:
# Building the channel map and loading the templates

channels = hx.Channels(hx.DEFAULT_CHANNEL_MAP)

template_path = '/home/ws/fm7040/helix/helix/plugins/event_rqs/signalformation_templates.npy'
templates = np.load(template_path, allow_pickle=True)
uv_template = templates[0]
qp_template = templates[1]

## Triggered events (triplet signals will often cause triggers at the moment)

In [ ]:
# change this to choose another event if the one below is a bad one by chance
for i in range(10):

    plt.figure(figsize=(10,16))
    artificial_baselines = np.arange(len(channels)) * 300  # adding artificial baselines to each channel to separate the channels on the plot
    plt.plot(events['channel_data'][i].T + artificial_baselines, lw=0.5, alpha=0.8, color='C0')  # plotting data in each channel

    event = fit_results[i]

    # plotting two-template fits in the vacuum channels
    for i_vac, i_ch in enumerate(channels.indices_of_type(hx.ChannelType.VACUUM)):
        # i_vac is the ordinal index of the vacuum channel
        # i_ch is the corrending ordinal index in the array of all channels
        fit = event['vacuum_channel_uv_amplitude'][i_vac] * np.roll(uv_template, event['vacuum_channel_uv_offset'][i_vac]) + \
              event['vacuum_channel_qp_amplitude'][i_vac] * np.roll(qp_template, event['vacuum_channel_qp_offset'][i_vac])
        x = np.arange(len(uv_template)) - hx.DEFAULT_ALLOWED_FIT_SHIFTS[0]  # currently, the FitResults plugin does not fit the entire event. It skips -hx.DEFAULT_ALLOWED_FIT_SHIFTS[0] samples.
        plt.plot(x, fit+artificial_baselines[i_ch], lw=1, alpha=0.8, color='C1')

    # plotting one-template fits in the submerged channels
    for i_sub, i_ch in enumerate(channels.indices_of_type(hx.ChannelType.SUBMERGED)):
        fit = event['submerged_channel_uv_amplitude'][i_sub] * np.roll(uv_template, event['submerged_channel_uv_offset'][i_sub])
        x = np.arange(len(uv_template)) - hx.DEFAULT_ALLOWED_FIT_SHIFTS[0]
        plt.plot(x, fit+artificial_baselines[i_ch], lw=1, color='C3')


    plt.xlabel('Time (samples)')
    plt.ylabel('Current (ADC units)')
    plt.show()